In [1]:
#from scipy.sparse.linalg import spsolve
#from scipy.sparse import csc_matrix
#import scipy.sparse as sparse
import scipy.linalg as linalg
import matplotlib.pyplot as plt
import numpy as np
import matplotlib as mpl

In [ ]:
def generateTriDiag(size, w):
    return np.diag(w*np.ones(size-1),-1) + np.diag(np.ones(size),0) + np.diag(w*np.ones(size-1),1)

def generateMat0(shuffle=False):
    Z = 1*np.eye(2*size) + np.triu(0.1*np.random.rand(2*size,2*size))
    if shuffle:
        Z = Z[:, np.random.permutation(Z.shape[1])]
        Z = Z + 2*np.eye(2*size)
    A = Z[:size,:size]
    B = Z[:size,size:] + 0.2*np.eye(size)
    Z[:size,size:] = B
    C = Z[size:,:size] + 0.2*np.eye(size)
    Z[size:,:size] = C
    D = Z[size:,size:]
    return A, B, C, D, Z

def generateMat1(w=0.1):
    A = generateTriDiag(size, w)
    B = np.eye(size)
    C = generateTriDiag(size, w)
    D = generateTriDiag(size, w)
    Z = np.zeros((2*size,2*size))
    Z[:size,:size] = A
    Z[:size,size:] = B
    Z[size:,:size] = C
    Z[size:,size:] = D
    return A, B, C, D, Z

########################################################

def solveP0(A, B, Lu, v, u, f1):
    return linalg.solve(A + Lu*np.eye(A.shape[0]), f1 - np.dot(B,v) + Lu*u)

def solveP1(A, B, C, D, Lv, u, v, f1, f2, AA, invAA):
    M = D - np.dot(C,np.dot(invAA,B)) + Lv*np.eye(A.shape[0])
    h = np.dot(A-AA,u)
    f = f2 - np.dot(C,np.dot(invAA,f1-h)) + Lv*v
    return linalg.solve(M, f)

########################################################

def method0(nit):
    print("method 0")
    u = np.zeros(A.shape[0])
    v = np.zeros(A.shape[0])
    res = []
    for i in range(nit):
        v = linalg.solve(D, f2 - C.dot(u))
        u = linalg.solve(A + Lv*np.eye(A.shape[0]), f1 - np.dot(B,v) + Lv*v)
        res.append(np.linalg.norm(sol - np.concatenate((u,v))))
    return u, v, np.array(res)
    
def method1(nit):
    print("method 1")
    
    AA = np.diag(A.diagonal(),0)
    invAA = np.diag(1/A.diagonal(),0)
    
    u = np.zeros(A.shape[0])
    v = np.zeros(A.shape[0])
    res = []
    for i in range(nit):
        v = solveP1(A, B, C, D, Lv, u, v, f1, f2, AA, invAA)
        u = solveP0(A, B,       Lu, v, u, f1)
        res.append(np.linalg.norm(sol - np.concatenate((u,v))))
    return u, v, AA, invAA, np.array(res)

def method2(nit):
    print("method 2")
    
    DD = np.diag(D.diagonal(),0)
    invDD = np.diag(1/D.diagonal(),0)

    u = np.zeros(A.shape[0])
    v = np.zeros(A.shape[0])
    res = []
    for i in range(nit):
        h = np.dot(D-DD,v)
        u = linalg.solve(A - np.dot(B, np.dot(invDD, C)) + Lu*np.eye(size), f1 - np.dot(B, np.dot(invDD, f2-h)) + Lu*u)
        v = linalg.solve(D, f2 - np.dot(C,u))
        res.append(np.linalg.norm(sol - np.concatenate((u,v))))
    return u, v, DD, invDD, np.array(res)

def methodGradSplit(nit, alpha=0.5):
    print("method grad")
    u = np.zeros(A.shape[0])
    v = np.zeros(A.shape[0])
    res = []
    AtA = np.dot(A.T,A)
    CtC = np.dot(C.T,C)
    AtB = np.dot(A.T,B)
    CtD = np.dot(C.T,D)
    BtA = np.dot(B.T,A)
    DtC = np.dot(D.T,C)
    BtB = np.dot(B.T,B)
    DtD = np.dot(D.T,D)
    for i in range(nit):
        u = u - alpha*2*(np.dot(AtA+CtC,u) + np.dot(AtB+CtD,v) - np.dot(A.T,f1) - np.dot(C.T,f2))
        v = v - alpha*2*(np.dot(BtA+DtC,u) + np.dot(BtB+DtD,v) - np.dot(B.T,f1) - np.dot(D.T,f2))
        res.append(np.linalg.norm(sol - np.concatenate((u,v))))
    return u, v, np.array(res)

def methodGradMonolithic(nit, alpha=0.2):
    uv = np.zeros(2*size)
    res = []
    for i in range(nit):
        uv = uv - alpha*2*np.dot(Z.T,np.dot(Z,uv)-f)
        res.append(np.linalg.norm(sol - uv))
    return uv, np.array(res)

In [ ]:
size = 12
A, B, C, D, Z = generateMat0(shuffle=False)
#A, B, C, D, Z = generateMat1()

f1 = np.random.rand(size)
f2 = np.random.rand(size)
print("determinant of Z =", linalg.det(Z))
print("condition number of Z =", np.linalg.cond(Z))

f = np.concatenate((f1,f2))
sol = linalg.solve(Z, f)
print(f"monolithic error = {np.linalg.norm(f-np.dot(Z,sol))}")

In [ ]:
fig, axs = plt.subplots(2, 4, figsize=(20,5))
cax1 = axs[0,0].matshow(A)
cax2 = axs[0,1].matshow(B)
cax3 = axs[1,0].matshow(C)
cax4 = axs[1,1].matshow(D)
cax5 = axs[0,2].matshow(AA)
cax6 = axs[1,2].matshow(invAA)
cax7 = axs[0,3].matshow(DD)
cax8 = axs[1,3].matshow(invDD)
fig.colorbar(cax1, ax=axs[0,0])
fig.colorbar(cax2, ax=axs[0,1])
fig.colorbar(cax3, ax=axs[1,0])
fig.colorbar(cax4, ax=axs[1,1])
fig.colorbar(cax5, ax=axs[0,2])
fig.colorbar(cax6, ax=axs[1,2])
fig.colorbar(cax7, ax=axs[0,3])
fig.colorbar(cax8, ax=axs[1,3])

fig1, ax = plt.subplots(1, 1, figsize=(10,5))
cax = ax.matshow(Z)
fig.colorbar(cax, ax=ax)

In [ ]:
Lu = 1.0
Lv = 0.0

u0, v0, res0 = method0(60)
u1, v1, AA, invAA, res1 = method1(60)
u2, v2, DD, invDD, res2 = method2(60)
#u3, v3, res3 = methodGradSplit(60,alpha=0.5)
#uv4, res4 = methodGradMonolithic(100,alpha=0.2)
uv5, res5 = jacobiStab(60, l=0.3)

plt.figure()
plt.plot(res0, label="method0")
plt.semilogy(res1, label="method1")
plt.plot(res2, label="method2")
#plt.plot(res3, label="method grad")
plt.plot(res5, label="method JacobiStab")

plt.grid()
plt.legend()
plt.show()